In [ ]:
import requests
import pandas as pd
import time
APP_TOKEN = "FCkWjoLQYcy4ao3B3s7u3t3tL7/D6okef9tqDNiru7c72tt3HsTtLeyp61FhjFGKiiUZ8VELQQALEEEjxgQ9QYgzio/EPEmkUE2MIiYISBBN3r0e5VgJIL7GJ989lvvnmm9/vN983MzsgU1p271DD0Jlyx6zi4QzIFDsc+GxQVlqyeK6zuLKkCOQ5OIYzd2Zcg86fqwxOkVNsOzRSmmpA94AiqwabNYYwU1dZjTMkg1U5BRos4tlYpLmJJbyATeka0nhNxtzR2hAWZHCK8ZOJBInzQgDHLat6IWaHFsICJB4URJrhRYYRBAFa/YZhwqhqIE5FIYwABOUBAQ/OdACGJWiWpL1+P96NueNQNyRNtVy8AAtn4bLZsXoe1stD5QwD6sgKgoWjkfpYayRaW9fSUeXLixXO6RBDHDKNya0aTYDuOCeb8PLTGFlvNmbyPDQMzBcen2FyUDZyAcw1wM9KzXA8DQlCDJL+IJ4QQEGkrNd0hUOXx2FbJMEjZl1ZqCIJpa+kqKVGohfyKNdqsUJEa9323/0mJ0uiBPUQVlcd6Yq0tWFhVUsm06aa8Agc4jxt7bUewU8TVJCjoQeHFE9RkM9NMh4pJ/GUWWo0VZBswQx3i4aqoYUYTtUFz9PFcmpVW/WIiGw0eX44uKAfGei2F3R8BU2UVO01hYolgjvbvLL6E6MR0qWEieBEhKkdWXlCGJdKSQI2tTObh7nUGTBCWBKhFOvz9ff3e/tJr6b3+AgAcN+q5qYYn4QKh+V87VofMKQrD/BIWSq8VaKWP4vSKQvLgJWnFgC1BwtTJAjQVE73ybDCU63/MORx9k2uhkJVR9BP0P4AA0maDOAkjReiOsK5BPXZOGCCS3sUTu+DKCVzPPTwVp6ZCtQlgSUpkSAZEXqEQFD0+IOi6ElQgjWZCCGAMJHgg8z/pUiuNs1jkNchKlieFyTHyRZ5bXVPPaevgEyfCaV6wdfZHufNCNlJx9XlZq+vuk1LrlkxAPyhq62ES5KvkSVLmQ5r/kIKYNf69EVo0AwEhWnRi/FaCrZpssSnZ9gC60Ibp6N0DMqyZZgWyUgqFS3cPl0Qev9ii7g2zoU9m/6Dc+mSrAw7XWcWK3u8YQXgUpLXPnm8vKb4NM66ctim1QZv17qFelq8Jeu2OqNYWyTH2UrC+DXTm6XsNdbyXh0amqlbN2xvq33z6tD6oGqdZUjXZBnqcXzatawoJuISMpxpRV2ABJe4GXbQ4jQFCJIiKHpavPjsMbp6pm1JhdyGbYOLvsqrtG/yR324KPvDBx0fg0HHB8UOB6gCd+GLwB2lzk6Xc06lISHolTjRa0g9qvWtqkNvH0ynOEkvvrnoy7lNwvqGpt8zCXPvytPLmKLyvDeF4QfBgolXhTInPjvviQHcdrGnBL/h1nKCAgGcAQxBk3Q3WHSx14Xf4pr3kcf3eebs6Cn+0HtHdwpPLaqQZByUTzg5HCVFrkFH0erntj/f92ZD5vEvzrUfadyQ+B4clQKblu3EiW/CFVvBEzd1bxytpKRXFi88ObLl0V2fjZ4/8mRw3/7DG85WDa3Z3Xzqp09qPlwwZ6Ha1Ts28suhV8XflhwsU1DZpy+8vKP17gWPvD8mHyg51rNtafr0toflx3B+z/B8bFaIf23kAFx2aO2e686fc39da44uH9ildGnH3tafOfvA2Mg9+/YersQq9s8aW7rjxNCcgz+m4ysr3vmz8dtVx/dcv+6+g5u+q68LnPhK/+OHl2Dj7ubbnQ8t2f76G40rBGz+upNPR0vnzVs/nNq898XyEmfSFVeePdP76+a/ju9Uh0f976LE+hsDb23p2q1s7FyzlRlfy78BbknKAu0RAAA="  # ← 본인 토큰으로 교체
HEADERS = {
    "Authorization": f"Bearer {APP_TOKEN}",
    "Content-Type": "application/json"
}
CATEGORY_ID = "26395"
MAX_ITEMS = 100 
def get_summary_items(category_id, limit=50, offset=0):
    url = "https://api.ebay.com/buy/browse/v1/item_summary/search"
    params = {"category_ids": category_id, "limit": limit, "offset": offset}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    return r.json().get("itemSummaries", [])
def get_item_details(item_id):
    url = f"https://api.ebay.com/buy/browse/v1/item/{item_id}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    return {}
def flatten(d):
    out = {}
    for k, v in d.items():
        if isinstance(v, dict):
            for subk, subv in v.items():
                out[f"{k}.{subk}"] = subv
        else:
            out[k] = v
    return out
all_data = []
for offset in range(0, MAX_ITEMS, 50):
    items = get_summary_items(CATEGORY_ID, 50, offset)
    for it in items:
        item_id = it.get("itemId")
        detail = get_item_details(item_id)
        flat = flatten(detail)
        all_data.append(flat)
        print(f":흰색_확인_표시: {item_id} 수집 완료")
        time.sleep(0.3) 
df = pd.DataFrame(all_data)
df.to_csv("ebay_health_beauty_detailed.csv", index=False)
print(":파일_폴더: 저장 완료: ebay_health_beauty_total_information.csv")

In [ ]:
import requests
import time
import csv
import base64
client_id = "nohhyunb-data-PRD-d47259a7e-1e5c55ec"
client_secret = "PRD-47259a7ec472-673a-4366-985a-7c59"

def get_token():
    resp = requests.post("https://api.ebay.com/identity/v1/oauth2/token",
                         headers={
                             "Content-Type": "application/x-www-form-urlencoded",
                             "Authorization": "Basic " + base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
                         },
                         data={
                             "grant_type": "client_credentials",
                             "scope": "https://api.ebay.com/oauth/api_scope"
                         })
    resp.raise_for_status()
    return resp.json()["access_token"]
token = get_token()
print(":흰색_확인_표시: Token")

def flatten(d, parent='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent}{sep}{k}" if parent else k
        if isinstance(v, dict):
            items += flatten(v, new_key, sep=sep).items()
        elif isinstance(v, list):
            if all(isinstance(x, dict) for x in v):
                for i, sub in enumerate(v):
                    items += flatten(sub, f"{new_key}[{i}]", sep=sep).items()
            else:
                items.append((new_key, str(v)))
        else:
            items.append((new_key, v))
    return dict(items)
    
def collect(category_id, total=5000, per_page=200, delay=0.5):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    rows = []
    for offset in range(0, total, per_page):
        params = {"limit": per_page, "offset": offset, "category_ids": category_id}
        url = "https://api.ebay.com/buy/browse/v1/item_summary/search"
        print(f":앞쪽_화살표: category_id={category_id}, offset={offset}")
        resp = requests.get(url, headers=headers, params=params)
        if resp.status_code != 200:
            print(":경고: 응답 코드:", resp.status_code)
            if resp.status_code == 429:
                print(f"Rate limit 초과 → {delay * 10}초 대기")
                time.sleep(delay * 10)
                continue
            break
        items = resp.json().get("itemSummaries", [])
        if not items:
            print(":흰색_확인_표시: 더 이상 데이터 없음, 중단")
            break
        for itm in items:
            rows.append(flatten(itm))
        time.sleep(delay)
    print(f":흰색_확인_표시: 수집된 아이템 수: {len(rows)}")
    return rows
    
CATEGORY_MAP = {
    "Bath & Body":                         "11838",
    "Fragrances":                          "11848",
    "Hair Care & Styling":                 "11854",
    "Health Care":                         "26395",
    "Makeup":                              "31786",
    "Medical & Mobility":                  "75035",
    "Nail Care, Manicure & Pedicure":      "11860",
    "Shaving & Hair Removal":              "31769",
    "Skin Care":                           "11863",
    "Vision Care":                         "31284",
    "Vitamins & Dietary Supplements":      "11776",
    "Wholesale Lots":                      "40044"
}

for name, cat_id in CATEGORY_MAP.items():
    print(f"\n:포장: {name.upper()} 수집 시작")
    rows = collect(category_id=cat_id)
    keys = sorted(set().union(*(r.keys() for r in rows)))
    fname = f"ebay_{name}_{len(rows)}.csv"
    with open(fname, "w", newline='', encoding='utf-8-sig') as f:
        wr = csv.DictWriter(f, fieldnames=keys)
        wr.writeheader()
        for r in rows:
            wr.writerow(r)
    print(f":글씨가_쓰여진_페이지: 저장 완료: {fname}")

### 상세컬럼 포함 코드

In [ ]:
import requests
import pandas as pd
import time


APP_TOKEN = "vKWUlpS29K5FmxQytzvbbru3e+zO0juDpF6ACPJH/GH0h6kiASFqSShIMIokCCQGUBogIYFoxBhIgPgRhQR19q6UayWA9BKbeH8u+8477zzPM+87X6BnXNmzG2s2/l7hHF/U2wN6ipxOthyUjSt9bmJx0bRSB8hzcPb2PNNTkin+ab4JE2pSaEJmUtdM5EolVM0UssYQZRmaoENTMQUNJpApYFGIRpbVCR4GCElDx7qoq5SrtipE8UD2BnkIWE5kPZCDxKrdjhnTQ1Tcw/HAhwIBb9wHfJxE2k3TQrWaiaGGQ5QHeHga+Gg2EAMBgfMILM94ObaVcrUgw1R0jbgwgApn4QrZvkYe1ntDhaaJDEyCUOHaSHW0IVJbtbg+Nt+dFys8qEMUQ2yZw78W6RJytUDVQvcexsx6C1FLFJFpUu5wboThQYXIbTAPAT8rdQAEJRSU/aLMiR4RoIJIWa0bCYjvjcO2KBItZ10FpGEFp++nKFEj3olEPPhVT0LUVrnsv+UWVBVZQUaIWlwZeSnS2EiFNb2jI21pcVqCGNKNTVW05PV7+CD0I5pFvMjzSBwcJBdpUOIRoyzSNUmxBTNd9TquRAQxGqmLJ08X4tSgNRgRGdto8vxYcFs/j6/VntDcDFq4Q7PnFCWICK7s5/3VH+qNsaHELYyGIoxsyMoTomAyqUjUyMZsHg6mTsoMUR0YJwW3u7u7m+nmGN1od3sAYN0vLquLih0oQQox52vXespU7t+BVrJURJJXxF/A6STBkiJ5SgBo7VSY54DPzw/qPhxWeKT1H4Y8zu7h1VCo6vBzbNDv9wNZjHu9MCAXojrCgwnqtnGgOEzTCWh0IZxUoYhokeSZlUCGIgkcL3u4gIxoyReUaW9Qluk4L5HBZIQAQvG4GAz8X4rkQdM8ikQD4YLleUFynKtX11S2V0NjKQp0WUipltzNTS2iFeGa/S3aEqvTXdmod6xemgLe0INWwl3JL1IVokyMjF9IAexaH70INbqJkTQqelFRT6JGXVXE9BibYENqhAZOR5GqEsOoSEaSydrCrdMFofcvloiH41zYvek/2Jfuysq003VssbL7myQATCqMvfMwop5w65AcOWzTKlO0a52gHhVvhZxWxxRrQjLHVpFyx0wmS5kx14iMgUzdMsgJm2mwT14xvQtpZC/Dhq6qyGhhR13LiYSFYVxFY62oC5DgChxjGy3r5wG5GfLc6HiJ2W101Vhbkgq5DNuGEv8DHqXdwy/1YUf2x2ach0HG+XmR0wnmgznsbDBrXHFzSfEj00wFI0aBMmMq7Rq5qxqI6ULpJFSMosmOkxPrpFdr6n7riVv7V/y6IOCoyHtT6G0DTw69KpQVs+V5Twxg+p2WUnbS1AoPD3xsAAQ4D8u3gtl3WkvYKSVPTNy2+Ovdxb492y61Xt++dcOyC7tr3gYVQ05OZ6mjJON0KOv7LsvfvrWiaWDejclbZtVmTtObD+4b2NFwkZ5yat+kj7as/XnT+NSxo3P7HPs/NuQbsP8D/5fv71/yysIZDR2nYz92rr3MdwoLni7bW91/s3Evc3PluuubvmpzCVdPt+wpn3Bi5fO9v0TnTmOP4akzf6BfWFd+5Oj5D/mym1d7j+4Q+z873+ZVp4Sv7EpW/3nr2pJT8y5PyKQufOcKHL/l/mPX7LOPHjrQNjBDefNI//hrSmLDuZMvl0lSqK/zqXf8y2ceujV97ZHm9z5d13du5uGdV4DrjWOfpGKPf3Pxtb9av9gZrzp+/Gzk0oHM9oVnBmrYcycmH9j8vXAGrKDmHHz3sfX06337tq7ehHJz+Tc2xuWd7REAAA=="          # ← eBay OAuth access-token 으로 교체
MAX_PER_CATEGORY = 5000               
LIMIT = 50                            
SLEEP_SUMMARY = 0.3                     
SLEEP_DETAIL  = 0.1                     

HEADERS = {
    "Authorization": f"Bearer {APP_TOKEN}",
    "Content-Type": "application/json"
}


CATEGORY_MAP = {
    "Bath & Body":                         "11838",
    "Fragrances":                          "11848",
    "Hair Care & Styling":                 "11854",
    "Health Care":                         "26395",
    "Makeup":                              "31786",
    "Medical & Mobility":                  "75035",
    "Nail Care, Manicure & Pedicure":      "11860",
    "Shaving & Hair Removal":              "31769",
    "Skin Care":                           "11863",
    "Vision Care":                         "31284",
    "Vitamins & Dietary Supplements":      "11776",
    "Wholesale Lots":                      "40044"
}


def get_summary_items(cat_id: str, limit: int = 50, offset: int = 0):
    """Browse API – item summaries"""
    url = "https://api.ebay.com/buy/browse/v1/item_summary/search"
    params = {"category_ids": cat_id, "limit": limit, "offset": offset}
    r = requests.get(url, headers=HEADERS, params=params)
    r.raise_for_status()
    return r.json().get("itemSummaries", [])

def get_item_details(item_id: str):
    """Browse API – single item detail"""
    url = f"https://api.ebay.com/buy/browse/v1/item/{item_id}"
    r = requests.get(url, headers=HEADERS)
    if r.status_code == 200:
        return r.json()
    return {}

def flatten(d: dict) -> dict:
    """중첩 딕셔너리를 1-level 딕셔너리로 평탄화"""
    out = {}
    for k, v in d.items():
        if isinstance(v, dict):
            out.update({f"{k}.{sk}": sv for sk, sv in v.items()})
        else:
            out[k] = v
    return out


rows = []

for name, cat_id in CATEGORY_MAP.items():
    print(f"📦 {name} ({cat_id}) 수집 시작")
    for offset in range(0, MAX_PER_CATEGORY, LIMIT):
        # --- 요약 호출 ---
        try:
            summaries = get_summary_items(cat_id, LIMIT, offset)
        except Exception as e:
            print(f"⚠️ 요약 실패 (offset {offset}): {e}")
            time.sleep(2)
            continue

        if not summaries: 
            break

        
        for s in summaries:
            item_id = s.get("itemId")
            try:
                detail = get_item_details(item_id)
            except Exception as e:
                print(f"   ⚠️ 상세 실패 ({item_id}): {e}")
                continue

            flat = flatten(detail)
            flat["categoryName"] = name   
            rows.append(flat)

            print(f"   ✅ {item_id} 수집, 총 {len(rows)}개")
            time.sleep(SLEEP_DETAIL)        

        time.sleep(SLEEP_SUMMARY)         


df = pd.DataFrame(rows).drop_duplicates("itemId")
output_file = "health_beauty_100k_detailed.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"\n✅ 최종 저장 완료: {len(df):,}개 → {output_file}")


CATEGORY_MAP = {
    "Bath & Body":                         "11838",
    "Fragrances":                          "11848",
    "Hair Care & Styling":                 "11854",
    "Health Care":                         "26395",
    "Makeup":                              "31786",
    "Medical & Mobility":                  "75035",
    "Nail Care, Manicure & Pedicure":      "11860",
    "Shaving & Hair Removal":              "31769",
    "Skin Care":                           "11863",
    "Vision Care":                         "31284",
    "Vitamins & Dietary Supplements":      "11776",
    "Wholesale Lots":                      "40044"
}